In [61]:
# Knowledge graph  for Detroit 911 call and charge data
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

import pandas as pd

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
police911call = '/home/nanta/MIDAS-Hackathon-Aug2025/1_detroit_open_data_portal/Police_Serviced_911_Calls_2024.csv'
RMS_crimefile = '/home/nanta/MIDAS-Hackathon-Aug2025/1_detroit_open_data_portal/RMS_Crime_Incidents_2024.csv'


In [68]:
df911 = pd.read_csv(police911call)
df911=df911[0:3000]

In [71]:

print(len(df911[df911['neighborhood_name']=='Nortown']))
df911[df911['neighborhood_name']=='Nortown'].head(5)

14


,X,Y,incident_entry_id,incident_id,incident_location,call_source,call_description,category,priority,call_group,...,total_response_time,total_time,zip_code,precinct,scout_car_area,neighborhood_name,council_district,longitude,latitude,ESRI_OID
3,-83.033012,42.438688,243662376,202436602376,E Lantz St & Concord St,W911,MALICIOUS DESTRUCTION I/P,MDPIP,3,11PCT,...,206.8,222.2,48234,11.0,1104,Nortown,3.0,-83.033012,42.438688,6004
31,-83.031961,42.438710,243662171,202436602171,E Lantz St & Helen St,W911,UDAA REPORT,UDAAREPT,4,11PCT,...,27.7,40.2,48234,11.0,1104,Nortown,3.0,-83.031961,42.438710,6034
266,-83.029695,42.433867,243660607,202436600607,E 7 Mile Rd & Rogge St,W911,TRAFFIC INCIDENT W/ INJURIES,TRAFF2,1,11PCT,...,9.6,22.4,48234,11.0,1104,Nortown,3.0,-83.029695,42.433867,6269
544,-83.024261,42.433978,243654780,202436504780,Van Dyke St & E 7 Mile Rd,W911,INVESTIGATE PERSON,INVPERS,3,11PCT,...,6.6,24.3,48234,11.0,1105,Nortown,3.0,-83.024261,42.433978,6564
774,-83.024326,42.436014,243653455,202436503455,Emery St & Van Dyke St,W911,ASSAULT AND BATTERY IP-JH,AB IP/JH,2,11PCT,...,50.5,55.7,48234,11.0,1104,Nortown,3.0,-83.024326,42.436014,6794


In [91]:
df_neighborhoods = pd.DataFrame({'neighborhood_name': df911['neighborhood_name'].unique()})
df_neighborhoods.head()
print(len(df_neighborhoods))

199


In [96]:
df_neighborhoods['neighborhood_name'] = df_neighborhoods['neighborhood_name'].fillna("None")

In [84]:
dfRMS = pd.read_csv(RMS_crimefile)
dfRMS=dfRMS[0:3000]
print(len(dfRMS[dfRMS['neighborhood']=='Nortown']))
dfRMS[dfRMS['neighborhood']=='Nortown'].head(5)

12


/tmp/ipykernel_44441/632846228.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  dfRMS = pd.read_csv(RMS_crimefile)


,X,Y,incident_entry_id,nearest_intersection,offense_category,offense_description,state_offense_code,arrest_charge,charge_description,incident_occurred_at,...,report_number,scout_car_area,police_precinct,census_block_2020_geoid,neighborhood,council_district,zip_code,longitude,latitude,ESRI_OID
920,-83.030818,42.435881,1080413-1301,Emery St & Carrie St,ASSAULT,ASSAULT AND BATTERY/SIMPLE ASSAULT,1301,13001,ASSAULT AND BATTERY/SIMPLE ASSAULT,2024/01/30 02:12:00+00,...,2401290387,1104,11,2.616351e+14,Nortown,3.0,48234.0,-83.030818,42.435881,21377
1434,-83.033012,42.438688,1145435-2605,E Lantz St & Concord St,FRAUD,FRAUD BY WIRE,2605,26005,FRAUD BY WIRE,2024/06/26 17:00:00+00,...,2406270137,1104,11,2.616351e+14,Nortown,3.0,48234.0,-83.033012,42.438688,33470
1874,-83.026471,42.438830,1095486-1301,Spencer St & E Lantz St,ASSAULT,ASSAULT AND BATTERY/SIMPLE ASSAULT,1301,13001,ASSAULT AND BATTERY/SIMPLE ASSAULT,2024/03/01 21:00:00+00,...,2403050275,1104,11,2.616351e+14,Nortown,3.0,48234.0,-83.026471,42.438830,42572
2078,-83.031866,42.435858,1091573-1302,Emery St & Helen St,AGGRAVATED ASSAULT,AGGRAVATED / FELONIOUS ASSAULT,1302,13002,AGGRAVATED / FELONIOUS ASSAULT ...,2024/02/25 23:23:00+00,...,2402250256,1104,11,2.616351e+14,Nortown,3.0,48234.0,-83.031866,42.435858,46108
2134,-83.029817,42.437385,1146517-1304,Rogge St & Bliss St,AGGRAVATED ASSAULT,NON-FATAL SHOOTING,1304,13004,NON-FATAL SHOOTING ...,2024/06/30 02:20:00+00,...,2406300010,1104,11,2.616351e+14,Nortown,3.0,48234.0,-83.029817,42.437385,46875


In [ ]:
len(dfRMS['neighborhood'].unique())

198

In [89]:
combined_unique_neighborhoods = set(df911['neighborhood_name'].unique()).union(set(dfRMS['neighborhood'].unique()))
print(f"Total unique neighborhoods in both datasets: {len(combined_unique_neighborhoods)}")

Total unique neighborhoods in both datasets: 203


In [105]:
only_in_911 = set(df911['neighborhood_name'].unique()) - set(dfRMS['neighborhood'].unique())
only_in_RMS = set(dfRMS['neighborhood'].unique()) - set(df911['neighborhood_name'].unique())
print(f"Unique neighborhoods only in 911 dataset: {len(only_in_911)}")
print(f"Unique neighborhoods only in RMS dataset: {len(only_in_RMS)}")

Unique neighborhoods only in 911 dataset: 5
Unique neighborhoods only in RMS dataset: 4


In [106]:
only_in_911

{'Carbon Works',
 'Garden Homes',
 'Hubbard Richard',
 'Oakwood Heights',
 'Tech Town'}

In [6]:
# read csv into json
def read_csv_to_json(file_path):
    import pandas as pd
    df = pd.read_csv(file_path)
    return df.to_dict(orient='records')

records911 = read_csv_to_json(police911call)
recordsRMS = read_csv_to_json(RMS_crimefile)
# Create a Neo4j graph

/tmp/ipykernel_44441/4096921373.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [34]:
print(len(records911), len(recordsRMS))

324871 88356


In [59]:
records911[3]



{'X': -83.033011583,
 'Y': 42.4386882580001,
 'incident_entry_id': 243662376,
 'incident_id': 202436602376,
 'incident_location': 'E Lantz St & Concord St',
 'call_source': 'W911 ',
 'call_description': 'MALICIOUS DESTRUCTION I/P',
 'category': 'MDPIP   ',
 'priority': '3',
 'call_group': '11PCT ',
 'call_code': '381030',
 'called_at': '2024/12/31 18:18:09+00',
 'intake_time': 2.2,
 'dispatch_time': 202.1,
 'travel_time': 2.5,
 'on_scene_time': 15.4,
 'total_response_time': 206.8,
 'total_time': 222.2,
 'zip_code': '48234',
 'precinct': 11.0,
 'scout_car_area': '1104',
 'neighborhood_name': 'Nortown',
 'council_district': 3.0,
 'longitude': -83.0330115832331,
 'latitude': 42.4386882577501,
 'ESRI_OID': 6004}

In [ ]:
records911[0].keys( )
'incident_location'

dict_keys(['X', 'Y', 'incident_entry_id', 'incident_id', 'incident_location', 'call_source', 'call_description', 'category', 'priority', 'call_group', 'call_code', 'called_at', 'intake_time', 'dispatch_time', 'travel_time', 'on_scene_time', 'total_response_time', 'total_time', 'zip_code', 'precinct', 'scout_car_area', 'neighborhood_name', 'council_district', 'longitude', 'latitude', 'ESRI_OID'])

In [ ]:
recordsRMS[0]
'incident_entry_id','charge_description','nearest_intersection', 'neighborhood', 'longitude', 'latitude'

{'X': -83.1964520935905,
 'Y': 42.3725927891856,
 'incident_entry_id': '1096475-4801',
 'nearest_intersection': 'Prest St & Plymouth Rd',
 'offense_category': 'OBSTRUCTING THE POLICE',
 'offense_description': 'OBSTRUCTING POLICE',
 'state_offense_code': 4801,
 'arrest_charge': '48000',
 'charge_description': 'OBSTRUCTING POLICE',
 'incident_occurred_at': '2024/03/07 21:49:00+00',
 'incident_time': '16:49:00',
 'incident_day_of_week': 4,
 'incident_hour_of_day': 16,
 'incident_year': 2024,
 'case_id': 1016852,
 'case_status': 'CLEARED BY ARREST',
 'case_status_updated_at': '2024/03/14 18:09:33+00',
 'updated_in_ibr_at': '2024/03/20 19:55:27+00',
 'updated_at': '2025/05/29 17:43:43.803+00',
 'crime_id': 4016852,
 'report_number': 2403070326,
 'scout_car_area': '0204',
 'police_precinct': '02',
 'census_block_2020_geoid': 261635353002024.0,
 'neighborhood': 'Plymouth-Hubbell',
 'council_district': 7.0,
 'zip_code': 48227.0,
 'longitude': -83.1964520935905,
 'latitude': 42.3725927891856,
 

In [17]:
records911[0]['X']

-83.024140691

In [42]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [39]:
record911_node_query = """
MERGE(incident:Record911 {incidentId: $record.incident_id})
    ON CREATE SET 
        incident.incident_time = $record.incident_time,
        incident.incident_location = $record.incident_location,
        incident.call_description = $record.call_description,
        incident.called_at = $record.called_at,
        incident.neighborhood_name = $record.neighborhood_name,
        incident.longitude = $record.longitude,
        incident.latitude = $record.latitude
RETURN incident
"""


In [31]:
kg.query(record911_node_query, 
         params={'record':records911[0]})

[{'incident': {'incidentId': 202436602386}}]

In [54]:
for record in records911[1200:3000]:
    kg.query(record911_node_query, 
             params={'record':record})

In [56]:
kg.query("""
         MATCH (n)
         RETURN count(n)
         """)

[{'count(n)': 3000}]

In [67]:
kg.query("""
    MATCH (incident:Record911)
    WHERE incident.neighborhood_name = "Nortown"
    RETURN count(incident) AS incident_count
""")

[{'incident_count': 14}]

In [73]:
recordsRMs_node_query = """
MERGE(crime:RecordRMS {incidentId: $record.incident_entry_id})
    ON CREATE SET 
        crime.charge_description = $record.charge_description,
        crime.nearest_intersection = $record.nearest_intersection,
        crime.neighborhood = $record.neighborhood,
        crime.longitude = $record.longitude,
        crime.latitude = $record.latitude
RETURN crime
"""
kg.query(recordsRMs_node_query, 
         params={'record':recordsRMS[0]})

[{'crime': {'nearest_intersection': 'Prest St & Plymouth Rd',
   'latitude': 42.3725927891856,
   'charge_description': 'OBSTRUCTING POLICE',
   'neighborhood': 'Plymouth-Hubbell',
   'incidentId': '1096475-4801',
   'longitude': -83.1964520935905}}]

In [78]:
for record in recordsRMS[0:3000]:
    kg.query(recordsRMs_node_query, 
             params={'record':record})

In [77]:
kg.query("""
    MATCH (incident:Record911), (crime:RecordRMS)
        WHERE incident.neighborhood_name = crime.neighborhood
    MERGE (incident)-[newrelationship:RELATED_TO]->(crime)
    RETURN count(newrelationship) AS incident_to_crime
""")

[{'incident_to_crime': 28}]

In [99]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Record911 {incidentId: INTEGER, incident_location: STRING, call_description: STRING, called_at: STRING, neighborhood_name: STRING, longitude: FLOAT, latitude: FLOAT}
RecordRMS {incidentId: STRING, longitude: FLOAT, latitude: FLOAT, charge_description: STRING, nearest_intersection: STRING, neighborhood: STRING}
Neighborhood {name: STRING}
Relationship properties:

The relationships:



In [97]:
# create nodes from combined_unique_neighborhoods with longitude and latitude
neighborhood_node_query = """
MERGE (n:Neighborhood {name: $neighborhood_name})
    ON CREATE SET 
        n.longitude = $longitude,
        n.latitude = $latitude
RETURN n
"""

for _, neighborhood in df_neighborhoods.iterrows():
    kg.query(neighborhood_node_query, 
             params={
                 'neighborhood_name': neighborhood['neighborhood_name'],
                 'longitude': neighborhood.get('longitude', None), 
                 'latitude': neighborhood.get('latitude', None)
             })

In [103]:
kg.query("""
    MATCH (incident:Record911), (crime:RecordRMS), (n:Neighborhood {name: "Nortown"})
        WHERE incident.neighborhood_name = n.name AND crime.neighborhood = n.name
    RETURN count(incident) AS incident_count, count(crime) AS crime_count
""")

[{'incident_count': 168, 'crime_count': 168}]

In [115]:
kg.query("""
    MATCH (incident:Record911), (n:Neighborhood {name: "Jefferson Chalmers"})
        WHERE incident.neighborhood_name = n.name 
    RETURN count(incident) AS incident_count
""")
# 'Garden Homes',
#  'Hubbard Richard',
#  'Oakwood Heights',

[{'incident_count': 14}]

In [118]:
# add relationship: [happened at] between incident and neighborhood 
kg.query("""
    MATCH (incident:Record911), (n:Neighborhood)
        WHERE incident.neighborhood_name = n.name 
    MERGE (incident)-[:HAPPENED_AT]->(n)
    RETURN count(incident) AS incident_count
""")

[{'incident_count': 2951}]

In [121]:
kg.query("""
    MATCH (incident:RecordRMS), (n:Neighborhood)
        WHERE incident.neighborhood = n.name 
    MERGE (incident)-[:HAPPENED_AT]->(n)
    RETURN count(incident) AS incident_count
""")

[{'incident_count': 2950}]

In [122]:
kg.refresh_schema()
print(kg.schema)

Node properties:
Record911 {incidentId: INTEGER, incident_location: STRING, call_description: STRING, called_at: STRING, neighborhood_name: STRING, longitude: FLOAT, latitude: FLOAT}
RecordRMS {incidentId: STRING, longitude: FLOAT, latitude: FLOAT, charge_description: STRING, nearest_intersection: STRING, neighborhood: STRING}
Neighborhood {name: STRING}
Relationship properties:

The relationships:
(:Record911)-[:HAPPENED_AT]->(:Neighborhood)
(:RecordRMS)-[:HAPPENED_AT]->(:Neighborhood)


In [124]:
kg.query("""
    MATCH (n:Neighborhood {name: $neighborhood_name})
    OPTIONAL MATCH (incident:Record911)-[:HAPPENED_AT]->(n)
    OPTIONAL MATCH (crime:RecordRMS)-[:HAPPENED_AT]->(n)
    RETURN n.name AS neighborhood, count(DISTINCT incident) AS incident_count, count(DISTINCT crime) AS crime_count
""", params={"neighborhood_name": "Nortown"})


[{'neighborhood': 'Nortown', 'incident_count': 14, 'crime_count': 12}]

In [125]:
kg.query("""
    MATCH (n:Neighborhood {name: $neighborhood_name})
    OPTIONAL MATCH (incident:Record911)-[:HAPPENED_AT]->(n)
    OPTIONAL MATCH (crime:RecordRMS)-[:HAPPENED_AT]->(n)
    RETURN n.name AS neighborhood, count(DISTINCT incident) AS incident_count, count(DISTINCT crime) AS crime_count
""", params={"neighborhood_name": "Jefferson Chalmers"})

[{'neighborhood': 'Jefferson Chalmers',
  'incident_count': 14,
  'crime_count': 23}]